In [8]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover
import math

In [2]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
data = yf.download(ticker, period="1y", interval="1h")


[*********************100%%**********************]  1 of 1 completed


In [ ]:
# , delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0

In [2]:
# Darwinex spreadsheet
df = pd.read_csv('../Data/Darwinex/GBPJPY60.csv', header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)


In [5]:
df = df.iloc[:10000]

In [10]:
df.head()

,Open,High,Low,Close,Volume,Candle
DateTime,,,,,,
2013-08-21 09:00:00,152.669,152.850,152.573,152.823,4543,Bull
2013-08-21 10:00:00,152.824,152.947,152.419,152.864,5086,Bull
2013-08-21 11:00:00,152.872,152.991,152.706,152.829,3732,Bear
2013-08-21 12:00:00,152.828,152.961,152.683,152.713,3534,Bear
2013-08-21 13:00:00,152.712,153.005,152.646,152.956,3639,Bull


In [9]:
conditions = [
    df.Close > df.Open,
    df.Close < df.Open
]

choices = ['Bull', 'Bear']

df['Candle'] = np.select(conditions, choices, default='Doji')

In [11]:
hh = float('-inf')
hl = float('-inf')

lh = float('inf')
ll = float('inf')

current_open = df.Open.iloc[-1]
current_high = df.High.iloc[-1]
current_low = df.Low.iloc[-1]
current_close = df.Close.iloc[-1]

previous_open = df.Open.iloc[-2]
previous_high = df.High.iloc[-2]
previous_low = df.Low.iloc[-2]
previous_close = df.Close.iloc[-2]

In [12]:
df.Close.iloc[5]

152.952

In [ ]:
# Uptrend tracer 

if df